아주 단순히 코싸인 유사도만으로 줄거리에 기반해서 영화를 추천하는 시스템을 만들어보고자 한다.

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
df = pd.read_csv('./data/movies_metadata.csv', low_memory=False)
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [12]:
# CPU로 실습을 진행하기 때문에 상위 2만개의 샘플만 사용하기로 함
data = df.head(20000)

In [13]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())


overview 열의 결측값의 수: 135


In [14]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

/var/folders/kv/nnq96swn52s8yvg0wjkr7sqw0000gn/T/ipykernel_4540/2009221054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview'] = data['overview'].fillna('')


In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview']) # raw document iterable을 넣으면 됨
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape) 
# (20000, 47487)
# TF-IDF 행렬의 크기는 20,000의 행을 가지고 47,847의 열을 가지는 행렬입니다. 다시 말해 20,000개의 영화를 표현하기 위해서 총 47,487개의 단어가 사용되었음을 의미합니다. 

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [17]:
tfidf.vocabulary_

{'led': 24361,
 'woody': 46617,
 'andy': 2051,
 'toys': 43131,
 'live': 24957,
 'happily': 18727,
 'room': 36185,
 'birthday': 4736,
 'brings': 5775,
 'buzz': 6290,
 'lightyear': 24759,
 'scene': 37166,
 'afraid': 1254,
 'losing': 25229,
 'place': 32291,
 'heart': 19047,
 'plots': 32424,
 'circumstances': 7974,
 'separate': 37778,
 'owner': 30780,
 'duo': 12920,
 'eventually': 14393,
 'learns': 24325,
 'aside': 2811,
 'differences': 11645,
 'siblings': 38509,
 'judy': 22536,
 'peter': 31851,
 'discover': 11888,
 'enchanted': 13728,
 'board': 5054,
 'game': 16798,
 'opens': 30240,
 'door': 12442,
 'magical': 25725,
 'world': 46664,
 'unwittingly': 44575,
 'invite': 21663,
 'alan': 1495,
 'adult': 1135,
 'trapped': 43304,
 'inside': 21286,
 '26': 430,
 'years': 46965,
 'living': 24973,
 'hope': 19854,
 'freedom': 16355,
 'finish': 15562,
 'proves': 33496,
 'risky': 35885,
 'running': 36455,
 'giant': 17265,
 'rhinoceroses': 35691,
 'evil': 14422,
 'monkeys': 28035,
 'terrifying': 42245,


In [16]:
# 이제 20,000개의 문서 벡터에 대해서 상호 간의 코사인 유사도를 구합니다.
# 이는 20,000개의 각 문서 벡터(영화 줄거리 벡터)와 자기 자신을 포함한 20,000개의 문서 벡터 간의 유사도가 기록된 행렬입니다.
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [18]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)


4


In [19]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]


In [21]:
# The Dark Knight Rises와 가장 유사한 영화들을 찾아와봐.
get_recommendations('The Dark Knight Rises')


12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object